<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/Soil_Sample_%7C_JaxEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8' # Use 8 CPU devices

In [1]:
import jax
import jax.lax
import numpy as np

In [2]:
FNV_PRIME = 0x01000193
def fnv(v1, v2):
    return ((v1 * FNV_PRIME) ^ v2) % 2**32

In [3]:
dataset = np.random.randint(low=0,high=9999,size=(10,255,32),dtype=np.uint32)
w = 255
mix = np.random.randint(low=0,high=9999,size=(32*2,10),dtype=np.uint32)
mixhashes = 4
n = mixhashes*12

In [4]:
mix = np.random.randint(low=0,high=9999,size=(10,32*2),dtype=np.uint32)
# for i in range(64):
#     p = fnv(i ^ 255, mix[:,i % 4]) % (n // mixhashes) * mixhashes
#     newdata = np.concatenate((dataset[p],dataset[p+1]),1)
#     # print(newdata.shape)
#     # for j in range(2):
#         # newdata.extend(dataset_lookup(p + j)))
#     mix = fnv(mix,newdata)#list(map(fnv, mix, newdata))
# mix.shape

In [5]:
def hasher(i,inp):
    # print(dataset_)
    mix_,dataset_=inp
    p = fnv(i ^ 255, mix_[i % 4]) % (n // mixhashes) * mixhashes
    newdata = jax.numpy.concatenate((dataset_[p],dataset_[p+1]))#jax.lax.concatenate((dataset_[p],dataset_[p+1]))
    mix_ = fnv(mix_,newdata)        
    return mix_,dataset_

In [6]:
def hasher2(i,inp):
    # print(dataset_)
    mix_,dataset_=inp
    p = fnv(i ^ 255, mix_[:,i % 4]) % (n // mixhashes) * mixhashes
    newdata = jax.numpy.concatenate((dataset_[p],dataset_[p+1]),1)#jax.lax.concatenate((dataset_[p],dataset_[p+1]))
    mix_ = fnv(mix_,newdata)        
    return mix_,dataset_

In [7]:
def body_funct(mix):
    global dataset
    ret = jax.lax.fori_loop(0, 32, hasher, (mix,dataset))
    return ret[0]

In [8]:
def body_funct2(mix,dataset):
    # global dataset
    ret = jax.lax.fori_loop(0, 32, hasher2, (mix,dataset))
    return ret[0]

In [9]:
from jax import device_put

In [10]:
import time

In [11]:
from jax import random, pmap


In [14]:
batch = 100000
mix = np.random.randint(low=0,high=9999,size=(batch,32*2),dtype=np.uint32)
dataset = np.random.randint(low=0,high=9999,size=(255,32),dtype=np.uint32)

mix = device_put(mix)
dataset = device_put(dataset)
%timeit jax.vmap(body_funct)(mix)
# dataset2 = np.random.randint(low=0,high=9999,size=(255,32),dtype=np.uint32)
# dataset2=device_put(dataset2)

# %timeit (body_funct2)(mix,dataset2).block_until_ready()

The slowest run took 4.75 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 5: 151 ms per loop


In [ ]:
# pmap(body_funct2)(mix,dataset2).block_until_ready()

ValueError: ignored

In [ ]:
batch/(267*(10**(-3)))

3745318.352059925